# Хакатон_2023

* загружаем ранее подготовленный датасет с собранной информацией из json + сущности из docx и датасет с текстами из документов<br>
* размечаем данные для обучения spacy новым NER<br>
* проверяем новые документы на наличие в них ошибок

Имеющаяся информация:
* id,
* 'ORG' - название организации,
* 'signatory' - подписант
* 'date_sig' - дата подписания,
* 'position_of_signatory' - должность подписанта,
* 'title' - название документа,
* 'Task1.task_text' - текст задачи,
* 'Task1.task_responsibles_people' - ответствнный за задачу,
* 'Task1.task_responsibles_groups' - ответственный отдел за задачу,
* 'Task1.task_deadline' - срок выполнения задачи, (и так по каждой задаче),
* 'Global_supervisor' - главный ответственный,
* 'Global_deadline' - общий срок выполнения

### Ипортируем необходимые библиотеки

In [1]:
! pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=3fab3fda23ee4a0e32bff0c94f46bb94c7194c9442def5b8715585129ff264d5
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [2]:
import os
import zipfile

import docx
from termcolor import colored

import re

# для работы с графами
import networkx as nx

import json
import pandas as pd

In [3]:
! pip install pullenti-wrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 4.3 MB/s eta 0:00:00


In [4]:
from pullenti_wrapper.processor import (
    Processor,
    ORGANIZATION,
    DATE,
    GEO,
    PERSON,
    MONEY,
    PHONE, 
    ADDRESS
)

In [5]:
!python -m spacy download ru_core_news_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-18 08:49:45.364964: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0c963fa33bdc43b039e5098775080deb857f28fcf3e048092d06363f3568b336
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
✔ Download and installation 

In [6]:
import spacy
nlp = spacy.load('ru_core_news_lg')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Загружаем подготовленные ранее данные для разметки

In [85]:
df1 = pd.read_csv("/content/markup_18.01.2023.csv")

In [99]:
df1.head(3)

,id,Signatory,Organization,Task1.task_text,Date_sig,Title,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Task2.task_text,...,responsibles_people_Task4.,responsibles_people_Task5.,responsibles_people_Task6.,responsibles_people_Task7.,responsibles_people_Task8.,responsibles_people_Task9.,responsibles_people_respon,Nasmes_global_supervisor,Names_global_supervisor,Posts_global_supervisors
0,0,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить книги. Срок завершения: 19 апр 15.,11.4.2013,Приказ,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,"['19 апр 15', 1429383600.0]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Кравцову И.Е.'],['специалиста первой категории']
1,1,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Провести анализ поставщиков для замены вышедше...,3.6.2013,Приказ,Ответственность за исполнение распоряжения воз...,Служба ИТ,"['24 октября 12', 1351015200.0]",Выполнить ремонт кабеля сети передачи данных м...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Кима Юрия Ивановича', 'Сергеева Андрея', 'Ко...","['специалиста третьей категории', 'системного ..."
2,2,генеральный директор Созонов С.В.,"ООО ""Мегасофт""",Провести рефакторинг программного обеспечения ...,8.10.2014,Приказ,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,"['24.12.20', 1608750000.0]",Провести рефакторинг программного обеспечения ...,...,технического писателя,специалистом третьей категории,специалиста третьей категории,NaN,NaN,NaN,NaN,NaN,"['Абаимовым Симоном', 'Ответственным', 'Абаимо...","['тимлидами проектной команды', 'тимлид проект..."


In [14]:
def clear_txt(x):
  txt = ""
  doc = nlp(x)
  for entity in doc:
      res = entity.lemma_
      txt += (" " + res)
  return txt


In [ ]:
nlp = spacy.load("/content/pipeline")

In [ ]:
nlp = spacy.blank("ru").from_disk("/content/pipeline")

### Проверка новых документов на наличие ошибок

In [134]:
! unzip file.zip -d "/content/drive/MyDrive/Хакатон_2023/drive-download-20230118T112934Z-001.zip"

unzip:  cannot find or open file.zip, file.zip.zip or file.zip.ZIP.


In [135]:
all_text = []
file_names = []

path_normalized = os.path.normpath("/content/test")
for dirpath, dirnames, filenames in os.walk(path_normalized):
    for file in filenames:
        file_names.append(int(file.rstrip('.docx')))
        text_doc = []
        doc = docx.Document(os.path.normpath(path_normalized + "/" + file))
        # собираем документ в список по абзацам и удаляем пустые абзацы
        for par in doc.paragraphs:
            if par.text != "":
                text_doc.append(par.text)
        txt = '\n'.join(text_doc)
        all_text.append(txt)
new_df = pd.DataFrame({"id": file_names, "text": all_text}, columns=["id", "text"])
new_df.set_index('id', inplace=True)
new_df.sort_index(inplace=True) # сортируем по индеку=id 
print(new_df.tail(5))

                                                    text
id                                                      
17563  ООО "Мегасофт"\n\n\nПриказ\nВ связи с подготов...
18344  ООО "Мегасофт"\n\n\nПриказ\nВ связи с окончани...
19196  ООО "Мегасофт"\n\n\nПриказ\nВ связи с окончани...
19877  ООО "Мегасофт"\n\n\nПриказ\nВ связи с подготов...
19969  ООО "Мегафон"\n\n\nПриказ\nВ связи с подготовк...


In [136]:
df2 = new_df.copy()

In [137]:
def clear_text(x):
    txt = ""
    s = re.sub("\s+", " ", x)
    doc = nlp(s)
    for entity in doc:
      if entity not in ""
        res = entity.lemma_
        txt += (" " + res)
    return txt

In [138]:
df2["clear_text"] = df2["text"].apply(lambda x: clear_text(x))

In [139]:
df2.head(3)

,text,clear_text
id,,
10008,"ООО ""Морковь""\n\n\nПриказ\nВ связи с окончание...","ооо "" морковь "" приказ в связь с окончание пр..."
10677,"ООО ""Максидом""\n\n\nПриказ\nВ связи с подготов...","ооо "" максидом "" приказ в связь с подготовка ..."
11727,"ООО ""Столы и стулья""\n\n\nПриказ\nДля организа...","ооо "" стол и стул "" приказ для организация ме..."


In [176]:
org_list = [''.join(i) for i in df1['Organization']]
org_list = [clear_text(i) for i in org_list]

In [ ]:
org_list

In [178]:
def verify_org(x):
       flag = "N"
       orgs = []
       doc = nlp(x)
       for entity in doc.ents:
          if entity.label_ == "ORG":
            orgs.append(entity.text)
       for i in orgs:
        if i in org_list:
          print(i)
          flag = "Y"
       return flag

In [179]:
def get_org(x):
       flag = "N"
       orgs = []
       doc = nlp(x)
       for entity in doc.ents:
          if entity.label_ == "ORG":
            orgs.append(entity.text)
       for i in orgs:
        if i in org_list:
          flag = "Y"
       return orgs

In [180]:
df2["verification_org"] = df2["clear_text"].apply(lambda x: verify_org(x))
df2["exist_org"] = df2["clear_text"].apply(lambda x: get_org(x))

In [167]:
persons_list =[''.join(i) for i in df1['executor_Task1.'] if i and isinstance(i, str)]
persons_list = [clear_text(i) for i in persons_list]

In [187]:
def verify_per(x):
       new_per = []
       persons = []
       doc = nlp(x)
       for entity in doc.ents:
          if entity.label_ == "PER":
            persons.append(entity.text)
       for i in persons:
        if i not in persons_list:
          print(111)
          new_per.append(i)

        return new_per

In [195]:
def exist_per(x):
       persons = []
       doc = nlp(x)
       for entity in doc.ents:
          if entity.label_ == "PER":
            persons.append(entity.text)
       existing_per = 0
       for i in persons:
          if i in persons_list:
            print(111)
            existing_per += 1

       return existing_per

In [196]:
df2["new_persons"] = df2["clear_text"].apply(lambda x: verify_per(x))
df2["exist_persons"] = df2["clear_text"].apply(lambda x: exist_per(x))

In [190]:
df2

,text,clear_text,verification_org,exist_org,new_persons,exist_persons
id,,,,,,
10008,"ООО ""Морковь""\n\n\nПриказ\nВ связи с окончание...","ооо "" морковь "" приказ в связь с окончание пр...",N,[с.в],[созонов],0.0
10677,"ООО ""Максидом""\n\n\nПриказ\nВ связи с подготов...","ооо "" максидом "" приказ в связь с подготовка ...",N,"[ооо "" максидом "", github]",[ромашкин],0.0
11727,"ООО ""Столы и стулья""\n\n\nПриказ\nДля организа...","ооо "" стол и стул "" приказ для организация ме...",N,"[ооо, с.с.галкина 2, поспелову н.а]",[никишин с.е],0.0
11747,"ООО ""Мегасофт""\n\n\nПриказ\nВ связи с окончани...","ооо "" мегасофт "" приказ в связь с окончание п...",N,"[ооо "" мегасофт "", с.в]",None,NaN
11989,"ООО ""Мегасофт""\n\n\nПриказ\nВ связи с окончани...","ооо "" мегасофт "" приказ в связь с окончание п...",N,"[ооо "" мегасофт ""]",[абаимов симон b )],0.0
12129,"ООО ""Мегасофт""\n\n\nПриказ\nДля организации ме...","ооо "" мегасофт "" приказ для организация мероп...",N,"[ооо "" мегасофт "", поспелову н.а. a.]",[сергеев андрей],0.0
12598,"ООО ""Мегасофт""\n\n\nПриказ\nВ связи с окончани...","ооо "" мегасофт "" приказ в связь с окончание п...",N,"[ооо "" мегасофт ""]",[абаимов симон],0.0
13024,"ООО ""Мегасофт-11""\n\n\nПриказ\nВ связи с оконч...","ооо "" мегасофт-11 "" приказ в связь с окончани...",N,"[ооо "" мегасофт-11 ""]",[кравцов с.с],0.0
13380,"ООО ""Мегасофт""\n\n\nПриказ\nДля организации ме...","ооо "" мегасофт "" приказ для организация мероп...",N,"[ооо "" мегасофт "", ит, ит, с.в]",[сергеев ален иванович],0.0


In [ ]:
df2.new_persons.value_counts()